## VWAP Momentum

In [26]:
from pyalgotrade import strategy
from pyalgotrade import plotter
from pyalgotrade.tools import yahoofinance
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import vwap
from pyalgotrade.stratanalyzer import returns
from pyalgotrade.stratanalyzer import sharpe
from pyalgotrade.stratanalyzer import drawdown
from pyalgotrade.stratanalyzer import trades
import VWAPM

import matplotlib.pyplot as plt 
#%matplotlib inline

# Load the yahoo feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("orcl", "data_daily.csv")

#def main(plot):
instrument = "orcl"
vwapWindowSize = 5
threshold = 0.01
plot=True

# Download the bars.
#    feed = yahoofinance.build_feed([instrument], 2011, 2012, ".")

strat = VWAPM.VWAPMomentum(feed, instrument, vwapWindowSize, threshold)
strat.run()

sharpeRatioAnalyzer = sharpe.SharpeRatio()
strat.attachAnalyzer(sharpeRatioAnalyzer)
# Attach different analyzers to a strategy before executing it.
retAnalyzer = returns.Returns()
strat.attachAnalyzer(retAnalyzer)
#sharpeRatioAnalyzer = sharpe.SharpeRatio()
#strat.attachAnalyzer(sharpeRatioAnalyzer)
drawDownAnalyzer = drawdown.DrawDown()
strat.attachAnalyzer(drawDownAnalyzer)
tradesAnalyzer = trades.Trades()
strat.attachAnalyzer(tradesAnalyzer)

if plot:
    plt = plotter.StrategyPlotter(strat, True, False, True)
    plt.getInstrumentSubplot(instrument).addDataSeries("vwap", strat.getVWAP())

plt.plot()


## Bollinger Bands

In [13]:
from pyalgotrade import strategy
from pyalgotrade import plotter
from pyalgotrade.tools import yahoofinance
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import bollinger
from pyalgotrade.stratanalyzer import sharpe


class BBands(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, bBandsPeriod):
        strategy.BacktestingStrategy.__init__(self, feed)
        self.__instrument = instrument
        self.__bbands = bollinger.BollingerBands(feed[instrument].getCloseDataSeries(), bBandsPeriod, 2)

    def getBollingerBands(self):
        return self.__bbands

    def onBars(self, bars):
        lower = self.__bbands.getLowerBand()[-1]
        upper = self.__bbands.getUpperBand()[-1]
        if lower is None:
            return

        shares = self.getBroker().getShares(self.__instrument)
        bar = bars[self.__instrument]
        if shares == 0 and bar.getClose() < lower:
            sharesToBuy = int(self.getBroker().getCash(False) / bar.getClose())
            self.marketOrder(self.__instrument, sharesToBuy)
        elif shares > 0 and bar.getClose() > upper:
            self.marketOrder(self.__instrument, -1*shares)


def main(plot):
    instrument = "orcl"
    bBandsPeriod = 40
    
    feed = yahoofeed.Feed()
    feed.addBarsFromCSV("orcl", "data_daily.csv")
    # Load the yahoo feed from the CSV file
#    feed = yahoofeed.Feed()
#    feed.addBarsFromCSV("orcl", "/home/nightrose/Documents/Github/try/xyz/data_90_96.csv.csv")
    # Download the bars.
#    feed = yahoofinance.build_feed([instrument], 2011, 2012, ".")

    strat = BBands(feed, instrument, bBandsPeriod)
    sharpeRatioAnalyzer = sharpe.SharpeRatio()
    strat.attachAnalyzer(sharpeRatioAnalyzer)

    if plot:
        plt = plotter.StrategyPlotter(strat, True, True, True)
        plt.getInstrumentSubplot(instrument).addDataSeries("upper", strat.getBollingerBands().getUpperBand())
        plt.getInstrumentSubplot(instrument).addDataSeries("middle", strat.getBollingerBands().getMiddleBand())
        plt.getInstrumentSubplot(instrument).addDataSeries("lower", strat.getBollingerBands().getLowerBand())

    strat.run()
    print "Sharpe ratio: %.2f" % sharpeRatioAnalyzer.getSharpeRatio(0.05)

    if plot:
        plt.plot()


if __name__ == "__main__":
    main(True)


DEBUG:broker.backtesting:Not enough cash to fill orcl order [19] for 2051 share/s
1996-01-17 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill orcl order [19] for 2051 share/s
DEBUG:broker.backtesting:Not enough cash to fill orcl order [20] for 2055 share/s
1996-01-18 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill orcl order [20] for 2055 share/s


Sharpe ratio: -0.60


## SMA Crossover

In [ ]:
from pyalgotrade import plotter
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.stratanalyzer import returns
import sma_crossover
import matplotlib.pyplot as plt 
#%matplotlib inline
# Load the yahoo feed from the CSV file
feed = yahoofeed.Feed()
feed.addBarsFromCSV("orcl", "data_96_3_10.csv")

# Evaluate the strategy with the feed's bars.
myStrategy = sma_crossover.SMACrossOver(feed, "orcl", 20)

# Attach a returns analyzers to the strategy.
returnsAnalyzer = returns.Returns()
myStrategy.attachAnalyzer(returnsAnalyzer)

# Attach the plotter to the strategy.
plt= plotter.StrategyPlotter(myStrategy)
# Include the SMA in the instrument's subplot to get it displayed along with the closing prices.
plt.getInstrumentSubplot("orcl").addDataSeries("SMA", myStrategy.getSMA())
# Plot the simple returns on each bar.
plt.getOrCreateSubplot("returns").addDataSeries("Simple returns", returnsAnalyzer.getReturns())

# Run the strategy.
myStrategy.run()
myStrategy.info("Final portfolio value: $%.2f" % myStrategy.getResult())

# Plot the strategy.
plt.plot()

INFO:strategy:Final portfolio value: $871219.61
1996-10-30 00:00:00 strategy [INFO] Final portfolio value: $871219.61
